## Classifier

Takes JSON input (from tree induction) and CSV file

In [2]:
import numpy as np
import pandas as pd
import json
import sys

In [21]:
def readArrange(filename):
    df = pd.read_csv(filename)
    aclass = df.iloc[1,0]
    labels = True
    if not isinstance(aclass, str):
        labels = False
    df = df.drop([0,1], axis=0)
    
    if labels:
        df = df[[c for c in df if c not in [aclass]] + [aclass]]
    return df, labels

In [20]:
# returns a pandas dataframe from the csvfile and a dictionary from the jsonfile
def readFiles(file1=None, file2=None):
    if file1 is None and file2 is None:
        if len(sys.argv) != 3:
            print("Not enough arguments.")
            exit(1)
        else:
            file1 = sys.argv[1]
            file2 = sys.argv[2]
    
    data, isLabeled = readArrange(file1)
    tree = None
    with open(file2) as f:
        tree = json.load(f)
    
    return data, tree, isLabeled

In [31]:
def traverseTree(row, tree, nodeType):
    if nodeType == "leaf":
        return tree["decision"]        
        
    elif nodeType == "node":
        val = row[tree["var"]]
        for obj in tree["edges"]:
            if obj["edge"]["value"] == val:
                newType = "leaf" if "leaf" in obj["edge"].keys() else "node"
                return traverseTree(row, obj["edge"][newType], newType)

In [100]:
def classify(d=None, t=None, labeled=False, confusion):
    numErrors = 0
    numCorrect = 0
    totalClassified = 0
    accuracy = 0
    errorRate = 0
    
    data = None
    tree = None
    if d is None and t is None:
        data, tree = readFiles("nursery.csv", "output.json")
    else:
        data = d
        tree = t
    
    out = []
    for i, row in data.iterrows():
        prediction = traverseTree(row, tree["node"], "node")
        
        # creating results dataframe as list of lists
        newLine = []
        for c in row:
            newLine.append(c)
        newLine.append(prediction)
        out.append(newLine)
        
        if labeled:
            actual = row[data.columns[-1]]
            if prediction != actual:
                numErrors += 1
            else:
                numCorrect += 1
            
        totalClassified += 1
    
    
    cols = [c for c in data.columns] + ["Prediction"]
    results = pd.DataFrame(out, columns=cols)

    if labeled:
        accuracy = numCorrect / totalClassified
        errorRate = numErrors / totalClassified
        
    print(totalClassified)
    
    
    
#     print(results)
#     print("Total Records Classifed: ", totalClassified)
#     print("Total Classified Correctly: ", numCorrect)
#     print("Total Classified Incorrectly: ", numErrors)
#     print("Accuracy: ", accuracy)
#     print("Error Rate: ", errorRate)
    
classify()

12960
            recommend  priority  not_recom  very_recom  spec_prior
recommend         0.0       2.0        0.0         0.0         0.0
priority          0.0    4266.0        0.0         0.0         0.0
not_recom         0.0       0.0     4320.0         0.0         0.0
very_recom        0.0     328.0        0.0         0.0         0.0
spec_prior        0.0    4044.0        0.0         0.0         0.0
